In [35]:
from PIL import Image
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import collections
import pickle
import argparse

from gensim.models.keyedvectors import KeyedVectors
from gensim.test.utils import get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec
from nltk.tokenize import word_tokenize
from tensorflow.contrib import rnn
import nltk
nltk.download('punkt')

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (6,6)
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing import sequence
import os
print(os.listdir("../"))

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
['nohup.out', '.zshrc', '.bash_logout', 'glove.6B.300d.txt', 'examples', '.ipython', 'README', '.ssh', '.config', '.conda', 'src', '.sudo_as_admin_successful', '.dl_binaries', '.bash_history', '.cmake', '.jupyter', 'ssl', '.local', '.cache', '.glove.6B.300d.txt.swp', 'Nvidia_Cloud_EULA.pdf', '.dlamirc', 'test_data.json', '.rnd', 'glove100.txt', '.viminfo', '.condarc', '.keras', 'test1.json', 'tf_stanford', '.bashrc', '.profile', 'anaconda2', '.ipynb_checkpoints', 'tutorials', 'nltk_data', 'anaconda3', 'glove.6B.50d.txt']


In [37]:
data = pd.read_json('~/test1.json', lines=True)
data.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


In [38]:
cates = data.groupby('category')
print("There are:", cates.ngroups, "category")
print(cates.size())

There are: 19 category
category
BLACK VOICES      55
BUSINESS           8
COMEDY            70
CRIME             23
EDUCATION          1
ENTERTAINMENT    237
IMPACT            11
LATINO VOICES      4
MEDIA             23
POLITICS         368
QUEER VOICES      52
RELIGION          11
SCIENCE            4
SPORTS            11
TECH              12
TRAVEL             8
WEIRD NEWS        27
WOMEN             19
WORLD NEWS        56
dtype: int64


In [39]:
data.category = data.category.map(lambda x: "WORLDPOST" if x == "THE WORLDPOST" else x)

In [40]:

# Change the text to tokens.
data['text'] = data.headline + " " + data.short_description
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data.text)
X = tokenizer.texts_to_sequences(data.text)
data['words'] = X

data['word_length'] = data.words.apply(lambda i: len(i))
data = data[data.word_length >= 5]

data.head()

,authors,category,date,headline,link,short_description,text,words,word_length
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,There Were 2 Mass Shootings In Texas Last Week...,"[201, 134, 219, 583, 909, 5, 127, 168, 109, 34...",27
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,Will Smith Joins Diplo And Nicky Jam For The 2...,"[37, 492, 1672, 2774, 6, 2775, 2776, 7, 1, 305...",20
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,Hugh Grant Marries For The First Time At Age 5...,"[1176, 2779, 2780, 7, 1, 55, 89, 12, 910, 2781...",25
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,"[307, 714, 2784, 2785, 584, 2786, 6, 220, 5, 1...",26
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",Julianna Margulies Uses Donald Trump Poop Bags...,"[2789, 2790, 914, 46, 10, 2791, 1179, 2, 425, ...",26


In [41]:
data.word_length.describe()

count    1000.000000
mean       25.413000
std         6.417446
min         8.000000
25%        21.000000
50%        25.000000
75%        30.000000
max        50.000000
Name: word_length, dtype: float64

In [42]:
maxlen = 50
X = list(sequence.pad_sequences(data.words, maxlen=maxlen))

In [43]:
categories = data.groupby('category').size().index.tolist()
category_int = {}
int_category = {}
# build category to int dict and int to category dict.
for i, k in enumerate(categories):
    int_category.update({i:k})
    category_int.update({k:i})

data['c2id'] = data['category'].apply(lambda x: category_int[x])

In [44]:
data.head()

,authors,category,date,headline,link,short_description,text,words,word_length,c2id
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,There Were 2 Mass Shootings In Texas Last Week...,"[201, 134, 219, 583, 909, 5, 127, 168, 109, 34...",27,3
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,Will Smith Joins Diplo And Nicky Jam For The 2...,"[37, 492, 1672, 2774, 6, 2775, 2776, 7, 1, 305...",20,5
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,Hugh Grant Marries For The First Time At Age 5...,"[1176, 2779, 2780, 7, 1, 55, 89, 12, 910, 2781...",25,5
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,"[307, 714, 2784, 2785, 584, 2786, 6, 220, 5, 1...",26,5
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",Julianna Margulies Uses Donald Trump Poop Bags...,"[2789, 2790, 914, 46, 10, 2791, 1179, 2, 425, ...",26,5


In [45]:
def clean_setence(text):
    text = re.sub(r"[^A-Za-z0-9(),!?\'\`\"]", " ", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"@[A-Za-z0-9]+", " ", text)
    text = text.strip().lower()

    return text

In [46]:

texts = data["text"]

words = list()
for test in texts:
    for word in word_tokenize(clean_setence(test)):
        words.append(word)

word_counter = collections.Counter(words).most_common()
word_dictionary = dict()
word_dictionary["<padding>"] = 0
word_dictionary["<unk>"] = 1
for word, _ in word_counter:
    word_dictionary[word] = len(word_dictionary)

with open("word_dict.pickle", "wb") as f:
    pickle.dump(word_dictionary, f)

reversed_dictionary = dict(zip(word_dictionary.values(), word_dictionary.keys()))
print(len(reversed_dictionary))
doc_max_len = 20

6194


In [47]:
print("Split data...")

Split data...


In [48]:
x = list(map(lambda d: word_tokenize(clean_str(d)), data["text"]))
x = list(map(lambda d: list(map(lambda w: word_dictionary.get(w, word_dictionary["<unk>"]), d)), x))
x = list(map(lambda d: d[:doc_max_len], x))
x = list(map(lambda d: d + (doc_max_len - len(d)) * [word_dictionary["<padding>"]], x))

y = list(data["c2id"])

In [49]:
train_x, valid_x, train_y, valid_y = train_test_split(x, y, test_size=0.10)

In [50]:
print("train and test set are prepared.")

train and test set are prepared.


In [55]:
class BidirectionalRNN(object):
    def __init__(self, reversed_dictionary, doc_max_len, num_class):
        self.x = tf.placeholder(tf.int32, [None, doc_max_len], name="x")
        self.x_len = tf.reduce_sum(tf.sign(self.x), 1)
        self.y = tf.placeholder(tf.int32, [None], name="y")
        self.keep_prob = tf.placeholder(tf.float32, [], name='keep_prob')
        self.global_step = tf.Variable(0, trainable=False)
        
        print("Run BidirectionalRNN...")
        self.vocabulary_size = len(reversed_dictionary)
        self.embedding_size = 300
        self.num_hidden = 100
        self.num_layers = 2
        self.learning_rate = 1e-3
        
        print("Loading Glove vectors...")
        glove_file = "../glove.6B.%dd.txt" % self.embedding_size
        word2vec_file = get_tmpfile("word2vec_format.vec")
        glove2word2vec(glove_file, word2vec_file)
        word_vectors = KeyedVectors.load_word2vec_format(word2vec_file)

        word_vec_list = list()
        for _, word in sorted(reversed_dictionary.items()):
            try:
                word_vec = word_vectors.word_vec(word)
            except KeyError:
                word_vec = np.zeros([self.embedding_size], dtype=np.float32)

            word_vec_list.append(word_vec)


        print("start embedding...")
        init_embeddings = tf.constant(np.array(word_vec_list), dtype=tf.float32)
        self.embeddings1 = tf.get_variable("embeddings1", initializer=init_embeddings, trainable=True)
        self.x_emb = tf.nn.embedding_lookup(self.embeddings1, self.x)

        fw_cells = [rnn.BasicLSTMCell(self.num_hidden) for _ in range(self.num_layers)]
        bw_cells = [rnn.BasicLSTMCell(self.num_hidden) for _ in range(self.num_layers)]
        fw_cells = [rnn.DropoutWrapper(cell, output_keep_prob=self.keep_prob) for cell in fw_cells]
        bw_cells = [rnn.DropoutWrapper(cell, output_keep_prob=self.keep_prob) for cell in bw_cells]

        self.rnn_outputs, _, _ = rnn.stack_bidirectional_dynamic_rnn(
            fw_cells, bw_cells, self.x_emb, sequence_length=self.x_len, dtype=tf.float32)
        self.last_output = self.rnn_outputs[:, -1, :]
        self.logits = tf.contrib.slim.fully_connected(self.last_output, num_class, activation_fn=None)
        self.predictions = tf.argmax(self.logits, -1, output_type=tf.int32)
        self.loss = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(logits=self.logits, labels=self.y))
        self.optimizer = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss, global_step=self.global_step)
        # make prediction
        correct_predictions = tf.equal(self.predictions, self.y)
        self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

In [52]:
def batch_iter(inputs, outputs, batch_size, epochs_num):
    inputs = np.array(inputs)
    outputs = np.array(outputs)

    num_batches_per_epoch = (len(inputs) - 1) // batch_size + 1
    for epoch in range(epochs_num):
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, len(inputs))
            yield inputs[start_index:end_index], outputs[start_index:end_index]

1


In [53]:
num_class = 41

In [58]:
tf.reset_default_graph()
with tf.Session() as sess:
    model = BidirectionalRNN(reversed_dictionary, doc_max_len, num_class)

    train_batches = batch_iter(train_x, train_y, 64, 10)
    num_batches_per_epoch = (len(train_x) - 1) // 64 + 1
    best_accuracy = 0
    
    sess.run(tf.global_variables_initializer())
    model_saver = tf.train.Saver(tf.global_variables())

    for x_batch, y_batch in train_batches:
        train_feed_dict = {
            model.x: x_batch,
            model.y: y_batch,
            model.keep_prob: 0.8
        }
        _, step, loss = sess.run([model.optimizer, model.global_step, model.loss], feed_dict=train_feed_dict)
        if step % 100 == 0:
            print("In step {0}, The loss is: {1}".format(step, loss))
        if step % 100 == 0:
            valid_batches = batch_iter(valid_x, valid_y, 64, 1)
            sum_accuracy, count = 0, 0
            for valid_x_batch, valid_y_batch in valid_batches:
                valid_feed_dict = {
                    model.x: valid_x_batch,
                    model.y: valid_y_batch,
                    model.keep_prob: 1.0
                }
                accuracy = sess.run(model.accuracy, feed_dict=valid_feed_dict)
                sum_accuracy += accuracy
                count += 1
            temp_accuracy = sum_accuracy / count

            print("\n Accuracy = {1}\n".format(step // num_batches_per_epoch, sum_accuracy / count))
            # Save the model that can produce the best accuracy
            if temp_accuracy > best_accuracy:
                best_accuracy = temp_accuracy
                model_saver.save(sess, "{0}/{1}.ckpt".format("saved_model", "naive"), global_step=step)
    
    print("end.")

Run BidirectionalRNN...
Loading Glove vectors...
start embedding...
In step 100, The loss is: 1.538771629333496

 Accuracy = 0.5677083432674408

end.
